In [ ]:
import jax.numpy as jnp

from sealagrangiax.grid import Dataset
from sealagrangiax.simulator import LinearSSC
from sealagrangiax.trajectory import Location


x0 = Location(jnp.zeros(2))
dt = jnp.asarray(6*60*60)
ts = jnp.linspace(0, dt*6, 7)

jax_ds = Dataset.from_arrays(
    variables={"u": jnp.ones((7, 100, 100)), "v": jnp.ones((7, 100, 100))},
    time=ts, latitude=jnp.linspace(-89, 89, 100), longitude=jnp.linspace(-179, 179, 100),
    convert_uv_to_dps=True
)

linear_sim = LinearSSC()

traj = linear_sim(jax_ds, x0, ts[1:-1], dt0=jnp.asarray(30*60))

In [ ]:
import equinox as eqx


class DummySimulator(eqx.Module):

    intercept
    slope

    __call__(self, ds, x0, ts, dt0):